### IMPORT DE BIBLIOTECAS

In [2]:
from collections import defaultdict
import pandas as pd
import numpy as np
import os

import implicit
import implicit.evaluation

from scipy.sparse import csr_matrix, save_npz, load_npz

from matplotlib import pyplot as plt
import seaborn as sns

# from surprise import Dataset, Reader, SVD, SVDpp, KNNWithMeans
# from surprise.model_selection import train_test_split, cross_validate
# from surprise import accuracy
# from surprise.model_selection import cross_validate

# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import pickle
# import flask
from tqdm import tqdm
from deep_translator import GoogleTranslator


### Carregamento de dados de categorias e histórico de clicks

In [2]:
df_de_cat = pd.read_csv(os.getcwd()+'\.txt\category_de.txt') # DataFrame Informativo das categorias

In [3]:
clicks_de = pd.read_csv(os.getcwd()+'\.txt\clicks_de_sample_2.txt', sep = ',', header=0) # df de histórico de clicks
clicks_de.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696606 entries, 0 to 696605
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   UserId       696606 non-null  object
 1   OfferId      696606 non-null  object
 2   OfferViewId  696606 non-null  object
 3   CountryCode  696606 non-null  object
 4   Category     696606 non-null  int64 
 5   Source       696606 non-null  object
 6   UtcDate      696606 non-null  object
 7   Keywords     2886 non-null    object
 8   OfferTitle   693966 non-null  object
dtypes: int64(1), object(8)
memory usage: 47.8+ MB


Valores missing na coluna de OfferTitle que serão excluídos.

In [4]:
clicks_de = clicks_de[clicks_de.OfferTitle.isna() == False]

### CRIAÇÃO DE COLUNA DE CLICKS POR CATEGORIA

In [5]:
clicks_de.drop(columns = ['Keywords'], axis = 1, inplace=True)  # DROP DAS KEYWORDS

df_de_cat.rename({'Ancertor_ID':'Ancestor_ID'}, axis = 1, inplace = True) # RENAME DO ANCESTOR_ID

df_de_cat.drop('Unnamed: 0', axis =1, inplace = True) # Remoção de coluna Unnamed

clicks_de['Cat_clicks'] = clicks_de.groupby('Category')['OfferId'].transform('count') # Criação de coluna de clicks por categoria

### Conversão das colunas de usuário e oferta em categórica e criando novas colunas com os códigos adotados 

In [6]:
clicks_de.UserId = clicks_de.UserId.astype('category')
clicks_de.OfferId = clicks_de.OfferId.astype('category')

In [7]:
clicks_de['User'] = clicks_de.UserId.cat.codes
clicks_de['Offer'] = clicks_de.OfferId.cat.codes

### Merge do Dataframe de categorias com o dataframe de clicks 

In [8]:
clicks_de = clicks_de.merge(df_de_cat, left_on = 'Category', right_on = 'ID').drop(['ID'], axis = 1) 

### Criação de coluna com o nº total de clicks do usuário e filtragem do dataframe apenas com usuários que clicaram 10 ou mais vezes em algum produto

In [9]:
clicks_de['UserTotalClicks'] = clicks_de.groupby(by=['User'])['OfferId'].transform('count')

In [10]:
#Cap minimo de clicks para integrar o sistema de recomendação
clicks_de_filtered = clicks_de[(clicks_de.UserTotalClicks > 10)]


### Agrupamento de dos clicks de usuário em ofertas únicas para termos a quantidade de cada usuário em cada oferta.


In [11]:
clicks_per_user_product = clicks_de_filtered.groupby(by=['User','Offer']).count()['UserTotalClicks'].reset_index().rename({'UserTotalClicks':'UserClicks'}, axis = 1)

In [12]:
clicks_de['ProductClicks'] = clicks_de.groupby(by='Offer')['OfferId'].transform('count')

### Criação de matrizes esparsas Usuário-item e item-usuário

In [13]:
alpha = 40
sparse_item_user = csr_matrix((clicks_per_user_product['UserClicks'], (clicks_per_user_product['Offer'], clicks_per_user_product['User'])))
#sparse_user_item = csr_matrix((clicks_per_user_product['UserClicks'].astype(float), (clicks_per_user_product['User'], clicks_per_user_product['Offer'])))
sparse_item_user = (sparse_item_user*alpha).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente
sparse_user_item = sparse_item_user.T.tocsr()

In [14]:
#Esparsidade de matriz
possible_interactions = sparse_item_user.shape[0]*sparse_item_user.shape[1]
interacted = len(sparse_item_user.nonzero()[0])
sparsity = 1 - interacted/possible_interactions
sparsity

0.9999994533909549

 Mais de 99% das interações possíveis entre usuários e produtos na atual base dados não foi ainda realizada. Segundo artigo: For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. Devemos reavaliar a matriz?

In [15]:
os.getcwd()

'C:\\Users\\gabri\\Desktop\\DH\\PI'

In [16]:
save_npz(os.getcwd()+"/.npz/sparse_user_item.npz", sparse_user_item)
save_npz(os.getcwd()+"/.npz/sparse_item_user.npz", sparse_item_user)

In [17]:
model_path = os.getcwd()+'/.pkl/de_als_model.pkl'

* Criação de diferentes matrizes esparsas para operar com o algoritmo. Usuário-item e item-usuário. Cada uma deve ser usada no momento preciso
* O alfa é o coeficiente de confiabilidade da interação do usuário com um item específico. Valor utilizado fi adotado com base no artigo: https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b. Mas, podemos testar outros valores na validação do modelo.
* Outro artigo de base pra elaboração do modelo: https://medium.com/analytics-vidhya/implementation-of-a-movies-recommender-from-implicit-feedback-6a810de173ac

# FUNÇÃO DE RECOMENDAÇÕES - IMPLICIT

## Treinamento de modelos

In [18]:
offers = pickle.load(open(os.getcwd()+"/.pkl/offers.pkl", "rb"))

Carregamentodo dicionário que converte os códigos de ofertas para o seu título de oferta. Ainda falta traduzir do alemão para o inglês para tirar mais significado dos resultados

In [19]:
def als_model():
    
    sparse_item_user = load_npz(os.getcwd()+"/.npz/sparse_item_user.npz")
      
    train, test, users_altered = ImplicitSupport.train_test_split(sparse_item_user, pct_test=0.2)

    model = implicit.als.AlternatingLeastSquares(factors=100, 
                                                 regularization=0.1, 
                                                 iterations=20, 
                                                 calculate_training_loss=False)
    
    model.fit(train)

    with open(model_path, 'wb') as pickle_out:
        pickle.dump(model, pickle_out)
    
    return train, test, model, users_altered

In [20]:
train, test, model, users_altered = als_model()

  0%|          | 0/20 [00:00<?, ?it/s]

In [27]:
user_vectors = model.user_factors
item_vectors = model.item_factors


In [91]:
def model_evaluation(train, test, model): 
    

    train, test = train.T.tocsr(), test.T.tocsr()
    
    p_at_k = implicit.evaluation.precision_at_k(model, 
                                                train_user_items=train, 
                                                test_user_items=test, 
                                                K=10, 
                                                show_progress = True)

    m_at_k = implicit.evaluation.mean_average_precision_at_k(model, 
                                                             train_user_items = train, 
                                                             test_user_items = test, 
                                                             K=10, 
                                                             show_progress = True)

    ndcg_at_k = implicit.evaluation.ndcg_at_k(model, 
                                              train_user_items = train,
                                              test_user_items = test, 
                                              K=10, 
                                              show_progress = True)

    auc_at_k = implicit.evaluation.AUC_at_k(model, 
                                            train_user_items = train, 
                                            test_user_items = test, 
                                            K=10, 
                                            show_progress = True)
    metrics = {'p@K':p_at_k, 
               'map@k': m_at_k, 
               'ndcg@k':ndcg_at_k, 
               'auc@k':auc_at_k}
    
    return metrics

* Sobre metricas de precisão @k: https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54
* Sobre NDCG: https://towardsdatascience.com/evaluate-your-recommendation-engine-using-ndcg-759a851452d1
* Sobre Mean Average Precision: https://towardsdatascience.com/breaking-down-mean-average-precision-map-ae462f623a52

In [ ]:
p, m, ndcg, auc = model_evaluation(als, sparse_item_user)

Ainda não foi realizada qualquer tunagem de hiperaparâmetros. Podemos pegar alguns valores de referencia para rodar um gridsearch

## Funções de recomendações

In [ ]:
def recommend(user):
    
    sparse_user_item = load_npz("sparse_user_item.npz")
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)
        
    recommended, _ = zip(*model.recommend(user, sparse_user_item))
    
    original_user_items = list(sparse_user_item[user_id].indices)

    return recommended, original_user_items

In [159]:
def most_similar_items(item_id, n_similar=10):
    '''computes the most similar items'''
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)

    similar, score = zip(*model.similar_items(item_id, n_similar)[1:])

    return similar

In [43]:
def most_similar_users(user_id, n_similar=10):
    '''computes the most similar users'''
    sparse_user_item = load_npz(os.getcwd()+"/.npz/sparse_user_item.npz")
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)

    similar, _ = zip(*model.similar_users(user_id, n_similar)[1:])

    # original users items
    original_user_items = list(sparse_user_item[user_id].indices)
    
    common_items_users = {}

    # now we want to add the items that a similar user has rated
    for user in similar:
        # Verifica em cada usuário considerado similar quais são os itens que estes
        # tem em comum com o usuário selecionado
        common_items_users[user] = set(list(sparse_user_item[user].indices)) & set(original_user_items)
    
    # retorna usuários similares, e quais são os itens comuns correspondentes a cada um desses usuários
    return similar, common_items_users

In [ ]:
def recalculate_user(user_ratings):
    '''adds new user and its liked items to sparse matrix and returns recalculated recommendations
       Receives the user clicked products vector (user_ratings)''' 

    alpha = 40
    m = load_npz('sparse_user_item.npz')
    n_users, n_movies = m.shape

    ratings = [alpha for i in range(len(user_ratings))]

    m.data = np.hstack((m.data, ratings))
    m.indices = np.hstack((m.indices, user_ratings))
    m.indptr = np.hstack((m.indptr, len(m.data)))
    m._shape = (n_users+1, n_movies)

    # recommend N items to new user
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)
        
    recommended, _ =  zip(*model.recommend(n_users, m, recalculate_user=True))
    
    return recommended

* A matriz m passa a ser a matriz com o novo usuário atualizado e é levada em consideração no para o cálculo de novos vetores.

Nota: 
* Após os ajustes na organização das matrizes esparsas, o modelo parece não mais repetir recomendações de itens que já foram clicados pelo usuário
* O modelo parece também não mais necessitar de tradução dos códigos de ofertas e usuário adotados na matriz esparsa para os códigos da matriz original

### Criação e armazenamento do dicionário código-titulo de oferta (carregado na parte de cima do código).

In [156]:
df_temp = clicks_de[['Offer','OfferTitle']]

In [161]:
df_temp = df_temp.drop_duplicates(['Offer','OfferTitle'])

In [169]:
teste_dicio = dict(zip(df_temp['Offer'], df_temp['OfferTitle']))

In [175]:
pickle.dump(offers, open('.pkl/offers.pkl', 'wb'))